In [1]:
query = '''
SELECT
  visitId 
  , STRING_AGG(p.productSKU ORDER BY h.hitNumber) AS product_sequence
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) as h, UNNEST(h.product) as p
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20160807'
  AND p.isClick IS NOT NULL
GROUP BY visitId
'''

In [2]:
import pandas as pd
from gensim.models import Word2Vec
from google.oauth2.service_account import Credentials

In [3]:
class ProductSequences:
    '''
    ProductSequences
    '''
    def __init__(self, df_column):
        df_column = df_column.copy()
        self.sequences = df_column.values.tolist()

    def __iter__(self):
        for sequence in self.sequences:
            yield sequence.split(',')

In [4]:
df = pd.read_gbq(query, project_id=credentials['project_id'],
                 credentials=Credentials.from_service_account_file('./private-key.json'))

print(df.info())

df.head()

NameError: name 'credentials' is not defined

In [ ]:
model = Word2Vec(ProductSequences(df.product_sequence), min_count=10, sg=1)

In [ ]:
embeddings_dict = dict(zip(model.wv.index2word, model.wv.vectors))